###### Imports and Settings

In [1]:
import pandas as pd
import numpy as np
import requests
import pickle
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)

+ population below poverty level 65+ 

In [2]:
#to read in... rb is read bite
with open('api_keys.pkl', 'rb') as keys_file:
        keys_dict_2 = pickle.load(keys_file)

In [3]:
api_key = keys_dict_2['CENSUS']

## API Calls

In [62]:
GNRC = ['161', #Stewart
       '125', #Montgomery
       '083', #Houston
       '085', #Humphreys
       '043', #Dickson
       '021', #Cheatham
       '147', #Robertson
       '165', #Sumner
       '037', #Davidson
       '189', #Wilson
       '169', #Trousdale
       '187', #Williamson
       '149'] #Rutherford

ID's 1, 6-11, 15-20, 28 - 35

In [63]:
data_appended = []
for i in GNRC:
    url_str= 'https://api.census.gov/data/2019/acs/acs5?key='+api_key
    predicates= {}
    get_vars= ["NAME", "GEO_ID",'B01001_001E','B01001_020E','B01001_021E','B01001_022E','B01001_023E','B01001_024E','B01001_025E','B01001_044E',
               'B01001_045E','B01001_046E','B01001_047E','B01001_048E','B01001_049E','B17001_001E','B17001_002E','B17001_014E','B17001_015E',
               'B17001_016E','B17001_028E','B17001_029E','B17001_030E']
    predicates["get"]= ",". join(get_vars)
    predicates["for"]= "county:{}".format(i)
    predicates["in"]= "state:47"
    data= requests.get(url_str, params= predicates)
    col_names = ['County','GEOID','pop','age_m_65to66','age_m_67to69','age_m_70to74','age_m_75to79','age_m_80to84','age_m_85+','age_f_65to66',
                 'age_f_67to69','age_f_70to74','age_f_75to79','age_f_80to84','age_f_85+','poverty_total_series','poverty_totalbelowpoverty',
                 'poverty_m55to64','poverty_m65to74','poverty_m75+','poverty_f55to64','poverty_f65to74','poverty_f75+','StateFIPS','CountyFIPS']
    data=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype = str)
    data['Region'] = 'GNRC'
    data_appended.append(data)
    
data_appended = pd.concat(data_appended)
data = data_appended.reset_index(drop = True)
print('Your API call is complete.')

Your API call is complete.


In [64]:
data.head()

,County,GEOID,pop,age_m_65to66,age_m_67to69,age_m_70to74,age_m_75to79,age_m_80to84,age_m_85+,age_f_65to66,age_f_67to69,age_f_70to74,age_f_75to79,age_f_80to84,age_f_85+,poverty_total_series,poverty_totalbelowpoverty,poverty_m55to64,poverty_m65to74,poverty_m75+,poverty_f55to64,poverty_f65to74,poverty_f75+,StateFIPS,CountyFIPS,Region
0,"Stewart County, Tennessee",0500000US47161,13427,192,194,399,175,155,94,210,263,362,236,257,95,13241,1821,135,78,12,84,103,89,47,161,GNRC
1,"Montgomery County, Tennessee",0500000US47125,200180,1506,2010,1792,1390,720,689,1531,1902,2606,1560,1209,1348,195756,26060,843,363,151,1221,493,439,47,125,GNRC
2,"Houston County, Tennessee",0500000US47083,8164,120,135,264,152,69,56,128,143,229,231,100,81,7882,1321,80,33,13,76,99,71,47,083,GNRC
3,"Humphreys County, Tennessee",0500000US47085,18435,213,328,444,414,131,76,285,285,514,427,206,233,17963,2806,143,55,44,262,91,179,47,085,GNRC
4,"Dickson County, Tennessee",0500000US47043,52680,645,731,930,628,478,259,679,824,1138,702,548,616,52063,7064,373,138,113,493,279,274,47,043,GNRC


## Prepare dataframe.

Drop unneeded ID's so that we can transpose and manipulate df easily.

In [65]:
test = data.drop(columns = ['StateFIPS','CountyFIPS']).reset_index(drop = True)

In [66]:
test.head()

,County,GEOID,pop,age_m_65to66,age_m_67to69,age_m_70to74,age_m_75to79,age_m_80to84,age_m_85+,age_f_65to66,age_f_67to69,age_f_70to74,age_f_75to79,age_f_80to84,age_f_85+,poverty_total_series,poverty_totalbelowpoverty,poverty_m55to64,poverty_m65to74,poverty_m75+,poverty_f55to64,poverty_f65to74,poverty_f75+,Region
0,"Stewart County, Tennessee",0500000US47161,13427,192,194,399,175,155,94,210,263,362,236,257,95,13241,1821,135,78,12,84,103,89,GNRC
1,"Montgomery County, Tennessee",0500000US47125,200180,1506,2010,1792,1390,720,689,1531,1902,2606,1560,1209,1348,195756,26060,843,363,151,1221,493,439,GNRC
2,"Houston County, Tennessee",0500000US47083,8164,120,135,264,152,69,56,128,143,229,231,100,81,7882,1321,80,33,13,76,99,71,GNRC
3,"Humphreys County, Tennessee",0500000US47085,18435,213,328,444,414,131,76,285,285,514,427,206,233,17963,2806,143,55,44,262,91,179,GNRC
4,"Dickson County, Tennessee",0500000US47043,52680,645,731,930,628,478,259,679,824,1138,702,548,616,52063,7064,373,138,113,493,279,274,GNRC


In [67]:
cols = ['pop','age_m_65to66','age_m_67to69','age_m_70to74','age_m_75to79','age_m_80to84','age_m_85+','age_f_65to66','age_f_67to69',
        'age_f_70to74','age_f_75to79','age_f_80to84','age_f_85+','poverty_total_series','poverty_totalbelowpoverty','poverty_m55to64','poverty_m65to74',
        'poverty_m75+','poverty_f55to64','poverty_f65to74','poverty_f75+']
test[cols] = test[cols].astype(int)

Transpose so that it's easy to calculate differences between the incorporated areas and the county to represent the unincorporated areas.

In [68]:
test = test.transpose()

In [69]:
test.columns = test.iloc[0]

In [70]:
test.head(10)

County,"Stewart County, Tennessee","Montgomery County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Dickson County, Tennessee","Cheatham County, Tennessee","Robertson County, Tennessee","Sumner County, Tennessee","Davidson County, Tennessee","Wilson County, Tennessee","Trousdale County, Tennessee","Williamson County, Tennessee","Rutherford County, Tennessee"
County,"Stewart County, Tennessee","Montgomery County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Dickson County, Tennessee","Cheatham County, Tennessee","Robertson County, Tennessee","Sumner County, Tennessee","Davidson County, Tennessee","Wilson County, Tennessee","Trousdale County, Tennessee","Williamson County, Tennessee","Rutherford County, Tennessee"
GEOID,0500000US47161,0500000US47125,0500000US47083,0500000US47085,0500000US47043,0500000US47021,0500000US47147,0500000US47165,0500000US47037,0500000US47189,0500000US47169,0500000US47187,0500000US47149
pop,13427,200180,8164,18435,52680,40181,70280,183437,687488,136666,10231,225389,315815
age_m_65to66,192,1506,120,213,645,594,775,2079,6150,1662,146,2326,2465
age_m_67to69,194,2010,135,328,731,654,1124,2647,7354,2177,108,2830,3243
age_m_70to74,399,1792,264,444,930,620,1072,3469,8858,2508,144,3533,3718
age_m_75to79,175,1390,152,414,628,486,882,2246,5640,1924,69,2057,2354
age_m_80to84,155,720,69,131,478,230,558,1255,3642,733,53,1435,1222
age_m_85+,94,689,56,76,259,126,244,995,2903,603,77,929,1197
age_f_65to66,210,1531,128,285,679,520,793,2134,7033,1788,207,2679,2493


Drop the first two rows.

In [71]:
test = test.loc[test['Stewart County, Tennessee'] != 'Stewart County, Tennessee']
test = test.loc[test['Stewart County, Tennessee'] != '0500000US47161']

In [72]:
test['Region'] = test['Stewart County, Tennessee']+test['Montgomery County, Tennessee']+test['Houston County, Tennessee']+test['Humphreys County, Tennessee']+test['Dickson County, Tennessee']+test['Cheatham County, Tennessee']+test['Robertson County, Tennessee']+test['Sumner County, Tennessee']+test['Davidson County, Tennessee']+test['Wilson County, Tennessee']+test['Trousdale County, Tennessee']+test['Williamson County, Tennessee']+test['Rutherford County, Tennessee']

In [73]:
test.head()

County,"Stewart County, Tennessee","Montgomery County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Dickson County, Tennessee","Cheatham County, Tennessee","Robertson County, Tennessee","Sumner County, Tennessee","Davidson County, Tennessee","Wilson County, Tennessee","Trousdale County, Tennessee","Williamson County, Tennessee","Rutherford County, Tennessee",Region
pop,13427,200180,8164,18435,52680,40181,70280,183437,687488,136666,10231,225389,315815,1962373
age_m_65to66,192,1506,120,213,645,594,775,2079,6150,1662,146,2326,2465,18873
age_m_67to69,194,2010,135,328,731,654,1124,2647,7354,2177,108,2830,3243,23535
age_m_70to74,399,1792,264,444,930,620,1072,3469,8858,2508,144,3533,3718,27751
age_m_75to79,175,1390,152,414,628,486,882,2246,5640,1924,69,2057,2354,18417


Now Transpose Back

In [74]:
data = test.transpose().reset_index()

In [75]:
data.head()

,County,pop,age_m_65to66,age_m_67to69,age_m_70to74,age_m_75to79,age_m_80to84,age_m_85+,age_f_65to66,age_f_67to69,age_f_70to74,age_f_75to79,age_f_80to84,age_f_85+,poverty_total_series,poverty_totalbelowpoverty,poverty_m55to64,poverty_m65to74,poverty_m75+,poverty_f55to64,poverty_f65to74,poverty_f75+,Region
0,"Stewart County, Tennessee",13427,192,194,399,175,155,94,210,263,362,236,257,95,13241,1821,135,78,12,84,103,89,GNRC
1,"Montgomery County, Tennessee",200180,1506,2010,1792,1390,720,689,1531,1902,2606,1560,1209,1348,195756,26060,843,363,151,1221,493,439,GNRC
2,"Houston County, Tennessee",8164,120,135,264,152,69,56,128,143,229,231,100,81,7882,1321,80,33,13,76,99,71,GNRC
3,"Humphreys County, Tennessee",18435,213,328,444,414,131,76,285,285,514,427,206,233,17963,2806,143,55,44,262,91,179,GNRC
4,"Dickson County, Tennessee",52680,645,731,930,628,478,259,679,824,1138,702,548,616,52063,7064,373,138,113,493,279,274,GNRC


## Calculations

###### Set cols to float for calculations

In [76]:
data[cols] = data[cols].astype(float)

#### Age

Add groups.

In [77]:
data = data.copy()
data['total65+'] = data['age_m_65to66']+data['age_m_67to69']+data['age_m_70to74']+data['age_m_75to79']+data['age_m_80to84']+data['age_m_85+']+data['age_f_65to66']+data['age_f_67to69']+data['age_f_70to74']+data['age_f_75to79']+data['age_f_80to84']+data['age_f_85+']

Drop input columns.

In [78]:
data = data.drop(columns = ['age_m_65to66','age_m_67to69','age_m_70to74',
                            'age_m_75to79','age_m_80to84','age_m_85+','age_f_65to66',
                            'age_f_67to69','age_f_70to74','age_f_75to79','age_f_80to84','age_f_85+'])

## Poverty

Add groups.

In [79]:
data['poverty65+'] = data['poverty_m65to74']+data['poverty_m75+']+data['poverty_f65to74']+data['poverty_f75+']

Calculate percentages.

In [80]:
data = data.copy()
data['inpoverty65+_%ofbelowpovertylevelpopulation'] = round((data['poverty65+'])*100/data['poverty_totalbelowpoverty'],2)
data['%inpoverty65+'] = round((data['poverty65+'])*100/data['total65+'],2)

Drop input columns.

In [81]:
data = data.drop(columns = ['poverty_total_series','poverty_totalbelowpoverty','poverty_m55to64','poverty_m65to74','poverty_m75+','poverty_f55to64',
                            'poverty_f65to74','poverty_f75+'])

# Export

In [82]:
data.head()

,County,pop,Region,total65+,poverty65+,inpoverty65+_%ofbelowpovertylevelpopulation,%inpoverty65+
0,"Stewart County, Tennessee",13427.0,GNRC,2632.0,282.0,15.49,10.71
1,"Montgomery County, Tennessee",200180.0,GNRC,18263.0,1446.0,5.55,7.92
2,"Houston County, Tennessee",8164.0,GNRC,1708.0,216.0,16.35,12.65
3,"Humphreys County, Tennessee",18435.0,GNRC,3556.0,369.0,13.15,10.38
4,"Dickson County, Tennessee",52680.0,GNRC,8178.0,804.0,11.38,9.83


In [83]:
#write to csv
data.to_csv('../data/michellerequest2019.csv')